In [4]:
import sklearn.decomposition
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sklearn.mixture import GaussianMixture
import plotly.express as px
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

This is only for the stock `stock_id=0`.

In [5]:
book_df = pd.read_parquet("./data/book_train.parquet/stock_id=0/c439ef22282f412ba39e9137a3fdabac.parquet")
trade_df = pd.read_parquet("./data/trade_train.parquet/stock_id=0/ef805fd82ff54fadb363094e3b122ab9.parquet")

target_df = pd.read_csv("./data/train.csv")
# the target_df contains all stocks. The following line restricts it to stock_id=0 and drops the label, since it's not relevant
target_df = target_df[target_df.stock_id == 0].drop('stock_id', axis=1)

FileNotFoundError: ignored

In [8]:
try:
    from google.colab import drive
    in_colab = True
except:
    in_colab = False

if in_colab:
    drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
book_df = pd.read_parquet("stock0_train.parquet")
trade_df = pd.read_parquet("stock0_trade.parquet")

target_df = pd.read_parquet("stock0_training.parquet")

In the following `book_df`, there are many 10-minute windows/buckets of data. The row number is meaningless. Each `time_id` corresponds to *one* bucket, and `seconds_in_bucket` is how far into that bucket the observation is. There are gaps in `time_id` and `seconds_in_bucket`. The gaps in `seconds_in_bucket` means that we do not have access to a complete second-by-second record of the order book. Many records are left out. In this dataframe, there are 8 features once the `time_id` and `seconds_in_bucket` have been specified. Note that `seconds_in_bucket` 0 is always present, but all other times may or may not be. In fact, I checked, and every second other than 0 is missing in at least one bucket. And every second from 0-599 is present in at least one bucket.

In [ ]:
book_df.head()

Plotting for book_df using only one time ID of 5.

In [ ]:
# Converting into dataframe
book_plot = pd.DataFrame(book_df)

# Select data for a specific time ID
time_data = book_plot[book_plot['time_id'] == 5]

# Create the line plot for bid_price1
plt.plot(time_data['seconds_in_bucket'], time_data['bid_price1'], label='Bid Price 1')

# Create the line plot for ask_price1
plt.plot(time_data['seconds_in_bucket'], time_data['ask_price1'], label='Ask Price 1')

# Set plot labels and title
plt.xlabel('Seconds in Bucket')
plt.ylabel('Price')
plt.title(f'Bid Price 1 and Ask Price 1 for Time ID')

# Add a legend
plt.legend()

# Set the DPI value for higher resolution
dpi = 1000

# Save the plot as a JPEG file
plt.savefig('book_plot.jpg', format='jpeg')

# Display the plot
plt.show()

The dataframe `trade_df` is very similar. Each row represents one trade. There are just 3 features here.

In [ ]:
trade_df.head()

In [ ]:
#Plotting for trade_df using only one time ID of 5.
# Select data for a specific time ID
time_data = trade_df[trade_df['time_id'] == 5]

# Create the line plot for price
plt.plot(time_data['seconds_in_bucket'], time_data['price'], label='Price')

# Create the line plot for size
plt.plot(time_data['seconds_in_bucket'], time_data['size'], label='Size')

# Create the line plot for order count
plt.plot(time_data['seconds_in_bucket'], time_data['order_count'], label='Order Count')

# Set plot labels and title
plt.xlabel('Seconds in Bucket')
plt.ylabel('Value')
plt.title('Trade Data for Time ID of 5')

# Add a legend
plt.legend()

# Set the DPI value for higher resolution
dpi = 1000

# Save the plot as a JPEG file
plt.savefig('trade_df.jpg', format='jpeg')

# Display the plot
plt.show()

Finally, the target data in `target_df` contains the actual realized volatility information , called `target`, from 10 minutes in the future. This is what we want to predict. The dataframe has all the `stock_id`s but we only care about `stock_id=0`. The `time_id` is again the bucket we care about. I'll restrict it to `stock_id=0` for convenience.

In [ ]:
target_df.head()

In [ ]:
#Data Exploration for target_df
#Converting into dataframe
target_plot = pd.DataFrame(target_df)

# Create the histogram
fig = go.Figure(data=[go.Histogram(x=target_plot['target'])])

# Set plot labels and title
fig.update_layout(
    xaxis_title='Target',
    yaxis_title='Frequency',
    title='Histogram of Target'
)

# Save the plot as a PNG file
offline.plot(fig, filename='target_plot.html', auto_open=False, image='png', image_filename='target_plot', image_width=800, image_height=600)

# Show the interactive plot
fig.show()

For the `book_df` dataframe, I am going to fill in the missing seconds with the data from the most recent second. For example, if is data at seconds 0, 4, and 7, then I will give seconds 1-3 the data from second 0, and seconds 5-6 the data from second 4.

In [ ]:
# get a list of all time buckets and seconds , without repetitions
unique_times = book_df['time_id'].drop_duplicates().to_frame()
unique_seconds = book_df['seconds_in_bucket'].drop_duplicates().sort_values().to_frame()

# take the cross of these. So all possible (bucket, second) pairs are present
full_book_df = unique_times.merge(unique_seconds, how='cross')

# filling in the seconds data requires things to be sorted by seconds
full_book_df = full_book_df.sort_values(['seconds_in_bucket', 'time_id'])
book_sorted_df = book_df.sort_values('seconds_in_bucket')

# This actually fills things in. 'on' means fill in seconds, 'by' means do it for each time_id.
full_book_df = pd.merge_asof(left=full_book_df, right=book_sorted_df, on='seconds_in_bucket', by='time_id')

# go back to our original order
full_book_df = full_book_df.sort_values(['time_id', 'seconds_in_bucket']).reset_index(drop=True)

Now we see that all seconds are present. There are changes at seconds 1, 5, 6.

In [ ]:
full_book_df.head(10)

In [ ]:
#Visualizing full_book_df
# Extract the relevant columns from the dataframe
time_id = full_book_df['time_id']
bid_price1 = full_book_df['bid_price1']
ask_price1 = full_book_df['ask_price1']
bid_size1 = full_book_df['bid_size1']
ask_size1 = full_book_df['ask_size1']

bid_price2 = full_book_df['bid_price2']
ask_price2 = full_book_df['ask_price2']
bid_size2 = full_book_df['bid_size2']
ask_size2 = full_book_df['ask_size2']

In [ ]:
#Plotting for Bid Price 1 and Ask Price 1
# Filter the data for time_id = 5
filtered_data = full_book_df[full_book_df['time_id'] == 5]

# Get the filtered bid_price1 and ask_price1 values
filtered_bid_price1 = filtered_data['bid_price1']
filtered_ask_price1 = filtered_data['ask_price1']

# Create the figure for Bid Price 1 and Ask Price 1
plt.figure()

# Plot the bid_price1 line
plt.plot(filtered_bid_price1, color='blue', label='Bid Price 1')

# Plot the ask_price1 line
plt.plot(filtered_ask_price1, color='green', label='Ask Price 1')

# Set plot labels and title
plt.xlabel('Index')
plt.ylabel('Price')
plt.title('Bid Price 1 and Ask Price 1 for Time ID 5')

# Add a legend
plt.legend()

# Set the DPI value for higher resolution
dpi = 1000

# Save the plot as a JPEG file
plt.savefig('bid_ask_price1.jpg', format='jpeg')

# Show the plot
plt.show()

Visualizing for bid_price2 and ask_price2. Plotting for Bid Price 2 and Ask Price 2

In [ ]:
# Filter the data for time_id = 5
filtered_data_2 = full_book_df[full_book_df['time_id'] == 5]

# Get the filtered bid_price1 and ask_price1 values
filtered_bid_price2 = filtered_data_2['bid_price2']
filtered_ask_price2 = filtered_data_2['ask_price2']

# Create the figure for Bid Price 1 and Ask Price 1
plt.figure()

# Plot the bid_price1 line
plt.plot(filtered_bid_price2, color='blue', label='Bid Price 1')

# Plot the ask_price1 line
plt.plot(filtered_ask_price2, color='green', label='Ask Price 1')

# Set plot labels and title
plt.xlabel('Index')
plt.ylabel('Price')
plt.title('Bid Price 2 and Ask Price 2 for Time ID 5')

# Add a legend
plt.legend()

# Set the DPI value for higher resolution
dpi = 1000

# Save the plot as a JPEG file
plt.savefig('bid_ask_price2.jpg', format='jpeg')

# Show the plot
plt.show()

In [ ]:
#Visualizing for bid size 1 and ask size 1
# Filter the data for time_id = 5
filtered_data_3 = full_book_df[full_book_df['time_id'] == 5]

# Get the filtered bid_price1 and ask_price1 values
filtered_bid_size1 = filtered_data_3['bid_size1']
filtered_ask_size1 = filtered_data_3['ask_size1']

# Create the figure for Bid Price 1 and Ask Price 1
plt.figure()

# Plot the bid_price1 line
plt.plot(filtered_bid_size1, color='blue', label='Bid Size 1')

# Plot the ask_price1 line
plt.plot(filtered_ask_size1, color='green', label='Ask Size 1')

# Set plot labels and title
plt.xlabel('Index')
plt.ylabel('Price')
plt.title('Bid Size 1 and Ask Size 1 for Time ID 5')

# Add a legend
plt.legend()

# Set the DPI value for higher resolution
dpi = 1000

# Save the plot as a JPEG file
plt.savefig('bid_ask_size1.jpg', format='jpeg')

# Show the plot
plt.show()

In [ ]:
#Visualizing for bid size 2 and ask size 2
# Filter the data for time_id = 5
filtered_data_4 = full_book_df[full_book_df['time_id'] == 5]

# Get the filtered bid_price1 and ask_price1 values
filtered_bid_size2 = filtered_data_4['bid_size2']
filtered_ask_size2 = filtered_data_4['ask_size2']

# Create the figure for Bid Price 1 and Ask Price 1
plt.figure()

# Plot the bid_price1 line
plt.plot(filtered_bid_size2, color='blue', label='Bid Size 2')

# Plot the ask_price1 line
plt.plot(filtered_ask_size2, color='green', label='Ask Size 2')

# Set plot labels and title
plt.xlabel('Index')
plt.ylabel('Price')
plt.title('Bid Size 2 and Ask Size 2 for Time ID 5')

# Add a legend
plt.legend()

# Set the DPI value for higher resolution
dpi = 1000

# Save the plot as a JPEG file
plt.savefig('bid_ask_size2.jpg', format='jpeg')

# Show the plot
plt.show()

Next, we just copy the code from the Optiver notebook to calculate the *weighted average price*, or `wap`. Since the true price is somewhere between the highest bid and lowest ask, this uses the number of bids and asks to determine where in the middle the true price is. We also, copying code from the Optiver notebook, calculate the log returns. We can do the same thing for the ordinary book. They'll be somewhat different.

In [ ]:
def log_return(list_stock_prices):
    return np.log(list_stock_prices).diff()

full_book_df['wap'] = (full_book_df['bid_price1'] * full_book_df['ask_size1'] +
                                full_book_df['ask_price1'] * full_book_df['bid_size1']) / (
                                       full_book_df['bid_size1']+ full_book_df['ask_size1'])

book_df['wap'] = (book_df['bid_price1'] * book_df['ask_size1'] +
                                book_df['ask_price1'] * book_df['bid_size1']) / (
                                       book_df['bid_size1']+ book_df['ask_size1'])

full_book_df.loc[:,'log_return'] = log_return(full_book_df['wap'])
# at time 0, this calculation is meaningless
full_book_df = full_book_df[full_book_df['seconds_in_bucket'] != 0]

book_df.loc[:,'log_return'] = log_return(book_df['wap'])
# at time 0, this calculation is meaningless
book_df = book_df[book_df['seconds_in_bucket'] != 0]

We can grab a specific bucket, given by `time_id==5`, and plot the stock price over the 10 minute window.

In [ ]:
# Plot the data
plt.plot(full_book_df[full_book_df["time_id"] == 5]['log_return'])

# Set plot labels and title
plt.xlabel('Data Point')
plt.ylabel('Log Return')
plt.title('Log Return for Time ID 5')

# Save the plot as a JPEG image
plt.savefig('log_return_plot.jpg', format='jpeg', dpi=300)

# Show the plot
plt.show()

In [ ]:
fig = px.line(book_df.loc[book_df['time_id']==5], x="seconds_in_bucket", y="log_return", title='log return of book:stock_id_0, time_id=5')
fig.update_layout(yaxis_range=[0.0015,-0.001])
fig.show()

In [ ]:
fig = px.line(full_book_df.loc[full_book_df['time_id']==5], x="seconds_in_bucket", y="log_return", title='log return of full book:stock_id_0, time_id=5')
fig.update_layout(yaxis_range=[0.0015,-0.001])
fig.show()

An important feature will be the realized volatility over the 10 minute window. We can compute this by taking
$$
    \sqrt{\sum_t r_t^2},
$$
where $r_t$ is the log return from time $t$ to $t+1$. I'll also add some basic features: max price, min price, average price, average `bid_size1/2`, average `ask_size1/2`, etc. See the code for details. The `lambda` functions are being used to compute volatility, and the mean/std/max of the gaps between orders in the trade dataframe.

In [ ]:
book_features_df = book_df.groupby('time_id').aggregate(
    vol = pd.NamedAgg('log_return', lambda x: np.sqrt(sum(a*a for a in x))), # anonymous function to compute vol from log returns
    mean_wap = pd.NamedAgg('wap', 'mean'),
    min_wap = pd.NamedAgg('wap', 'min'),
    max_wap = pd.NamedAgg('wap', 'max'),
    mean_bid1 = pd.NamedAgg('bid_size1', 'mean'),
    mean_bid2 = pd.NamedAgg('bid_size2', 'mean'),
    mean_ask1 = pd.NamedAgg('ask_size1', 'mean'),
    mean_ask2 = pd.NamedAgg('ask_size2', 'mean'),
    min_bid1 = pd.NamedAgg('bid_size1', 'min'),
    min_ask1 = pd.NamedAgg('ask_size1', 'min'),
)

trade_features_df = trade_df.groupby('time_id').aggregate(
    avg_gap = pd.NamedAgg('seconds_in_bucket', lambda x: np.mean(np.diff(x))),
    std_gap = pd.NamedAgg('seconds_in_bucket', lambda x: np.std(np.diff(x))),
    max_gap = pd.NamedAgg('seconds_in_bucket', lambda x: np.max(np.diff(x))),
    sum_size = pd.NamedAgg('size', 'sum'),
    max_size = pd.NamedAgg('size', 'max'),
    sum_orders = pd.NamedAgg('order_count', 'sum'),
    max_orders = pd.NamedAgg('order_count', 'max'),
)

features_df = book_features_df.join(trade_features_df, on='time_id')
features_df.head()

Now let's add the Fourier transforms. We'll do PCA on top of this. The benefit of the Fourier transform is to make things shift-independent. So if a pattern happens at times 100-110, the same pattern at times 220-230 can get picked up. To do this, I'm going to put the dataframe into a numpy tensor so that we can do the Fourier transform in parallel. I'll leave it in the tensor for now for the sake of PCA.

In [ ]:
num_times = len(full_book_df['time_id'].unique())
num_seconds = len(full_book_df['seconds_in_bucket'].unique())
# the below could be useful
# original_col_names = list(full_book_df.columns)
# fourier_col_names = ["fft_" + name for name in original_col_names if name not in ('time_id', 'seconds_in_bucket')]
# new_col_names = original_col_names + fourier_col_names

book_tensor = full_book_df.drop(['time_id', 'seconds_in_bucket'], axis=1).to_numpy(dtype='float').reshape(num_times, num_seconds, -1)
fourier = np.fft.fft(book_tensor, axis=1)

# now fourier has complex entries. This isn't so nice for working with, so we'll split the complex numbers into real and imaginary
real, imag = np.real(fourier), np.imag(fourier)
fourier = np.concatenate([real, imag], axis=-1)
print(f"Fourier shape: {fourier.shape}. This is datapoints, timesteps, features. Features has doubled since we have real + imaginary")

At this point, if we do PCA, we'll end up leaking data from the validation set. We also need to standardize our variables (again, on the training set only). So we really need to do some train/test split.

In [ ]:
train_proportion = .8
train_cutoff_index = int(train_proportion*fourier.shape[0])

fourier_train, fourier_test = np.split(fourier, [train_cutoff_index,])
train_mean = np.mean(fourier_train, axis=(0, 1))
train_std = np.std(fourier_train, axis=(0, 1))
normalized_train = (fourier_train-train_mean)/train_std
# we use the train_std and train_mean since that's what we do PCA on.
normalized_fourier = (fourier-train_mean)/train_std

Now we can run PCA on the train dataset. It takes my computer about 10 seconds, so it's not too slow. If this runs out of memory, we can switch to incremental PCA. I'll keep 10 dimensions (10 was selected by an elbow method).

In [ ]:
n_components = 10
fourier_pca = sklearn.decomposition.PCA(n_components)
fourier_pca.fit(normalized_train.reshape([normalized_train.shape[0], -1]))

# if we want to visualize the amount of variance explained, we can increase n_components and then plot the following
# plt.plot(fourier_pca.explained_variance_ratio_)

PCA Explained Variance Ratio Plot

In [ ]:
# Get the explained variance ratio
explained_variance_ratio = fourier_pca.explained_variance_ratio_

# Calculate the cumulative explained variance
cumulative_explained_variance = np.cumsum(explained_variance_ratio)

# Plot the explained variance ratio
plt.plot(range(1, n_components + 1), explained_variance_ratio, marker='o')
plt.xlabel('Principal Components')
plt.ylabel('Explained Variance Ratio')
plt.title('Explained Variance Ratio for Fourier PCA')
plt.grid(True)
plt.savefig('explained_variance_ratio.jpeg', dpi=300)
plt.show()

# Plot the cumulative explained variance
plt.plot(range(1, n_components + 1), cumulative_explained_variance, marker='o')
plt.xlabel('Principal Components')
plt.ylabel('Cumulative Explained Variance')
plt.title('Cumulative Explained Variance for Fourier PCA')
plt.grid(True)
plt.savefig('cumulative_explained_variance.jpeg', dpi=300)
plt.show()

Finally, we want to capture these components as features.

In [ ]:
pca_features = fourier_pca.transform(normalized_fourier.reshape([normalized_fourier.shape[0], -1]))

indices = full_book_df['time_id'].unique().reshape(-1, 1)
train_indices = indices[:train_cutoff_index].reshape(-1, 1)
test_indices = indices[train_cutoff_index:].reshape(-1, 1)

pca_col_names = ['time_id'] + ['pca_' + str(i) for i in range(n_components)]
pca_features_df = pd.DataFrame(data=np.concatenate((indices, pca_features), axis=1),  columns=pca_col_names)

# change time_id back to integer and make it an index
pca_features_df = pca_features_df.astype({'time_id': int})
pca_features_df = pca_features_df.set_index('time_id')

PCA Scatter Plot

In [ ]:
# Create a scatter plot
plt.figure(figsize=(10, 5))
plt.scatter(pca_features[:, 0], pca_features[:, 1], alpha=0.5)
plt.title('PCA Scatter Plot')
plt.xlabel('PC1')
plt.ylabel('PC2')
plt.savefig('pca_scatter_plot.jpeg', dpi=300)
plt.show()

We can combine this with the features we found earlier. Then we'll split into train and test (again).

In [ ]:
features_df = features_df.join(pca_features_df)
train_df = features_df.iloc[:train_cutoff_index, :]
test_df  = features_df.iloc[train_cutoff_index:, :]

target_train_df = target_df.iloc[:train_cutoff_index, :]
target_test_df = target_df.iloc[train_cutoff_index:, :]

Now `train_df` and `test_df` contain all the PCA features and the handcrafted features. The corresponding `target_train_df` and `target_test_df` contain the volatility that we actually want to predict. Next, we implement the GMM, which can give class probabilities as a feature.

In [ ]:
max_components = 10  # Adjust the max number of components as desired
component_range = range(1, max_components+1)

# Initialize BIC scores
bic_scores = []

# Iterate over different values of n_components
for n in component_range:
    # Create and fit the GMM model
    gmm = GaussianMixture(n_components=n)
    gmm.fit(train_df)

    # Calculate the BIC score
    bic = gmm.bic(train_df)

    # Store the BIC score
    bic_scores.append(bic)

# Find the optimal number of components that minimizes BIC
gmm_num_components = component_range[np.argmin(bic_scores)]
print("Optimal number of n components:", gmm_num_components)

Visualizing GMM

In [ ]:
# Create the line plot for BIC scores
plt.plot(component_range, bic_scores, marker='o')

# Set plot labels and title
plt.xlabel('Number of Components')
plt.ylabel('BIC Score')
plt.title('BIC Score vs Number of Components')

# Display the plot
plt.show()

Now that we have the optimal number of components, we can fit. Then we'll predict (on the test and train set, to get features) and add these probabilities to our features. We'll also drop one of the cluster probabilities from the feature list, since the final probability can just be computed from the others, since the probabilities must sum to one.

In [ ]:
gmm = GaussianMixture(n_components=gmm_num_components)
gmm.fit(train_df)
probabilities = gmm.predict_proba(features_df)
reduced_probabilities = np.delete(probabilities, 0, axis=1)

indices = full_book_df['time_id'].unique().reshape(-1, 1)

gmm_col_names = ['time_id'] + ['gmm_' + str(i) for i in range(gmm_num_components-1)]
gmm_features_df = pd.DataFrame(data=np.concatenate((indices, reduced_probabilities), axis=1),  columns=gmm_col_names)

# change time_id back to integer and make it an index
gmm_features_df = gmm_features_df.astype({'time_id': int})
gmm_features_df = gmm_features_df.set_index('time_id')

Finally, combine all features into a single dataframe. Split into test and train as before.

In [ ]:
features_df = features_df.join(gmm_features_df)
train_df = features_df.iloc[:train_cutoff_index, :]
test_df  = features_df.iloc[train_cutoff_index:, :]
features_df.head()

Initialize the Random Forest model. We are using the RandomForestRegressor class from sklearn.ensemble module.

In [ ]:
rf = RandomForestRegressor(random_state=42)

Define the grid of hyperparameters to search. We define a dictionary where the keys are the names of the hyperparameters we want to tune, and the values are lists of the values we want to try out.

In [ ]:
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10]
}

Set up the grid search. We are using the GridSearchCV class from sklearn.model_selection module. This class performs an exhaustive search over the specified parameter values for an estimator.

In [ ]:
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=3, scoring='neg_mean_squared_error')

Train the model. We are fitting the grid search object to the training data. This will perform the grid search and fit the best model to the data.

In [ ]:
grid_search.fit(train_df, target_train_df['target'])

Print the best parameters. We are printing the best hyperparameters found by the grid search.

In [ ]:
print("Best parameters: ", grid_search.best_params_)

Make predictions with the best model. We are getting the best estimator found by the grid search and using it to make predictions on the testing data.


In [ ]:
best_rf = grid_search.best_estimator_
predictions = best_rf.predict(test_df)

Calculate RMSE.


In [ ]:
rmse = np.sqrt(mean_squared_error(target_test_df['target'], predictions))
print("RMSE: ", rmse)

Calculate MAE.


In [ ]:
mae = mean_absolute_error(target_test_df['target'], predictions)
print("MAE: ", mae)

Calculate R-squared. This metric provides an indication of the goodness of fit of the model.


In [ ]:
r2 = r2_score(target_test_df['target'], predictions)
print("R-squared: ", r2)

Random Forest Visualization. First, we have to get the most important features to do this.

In [ ]:
# Get feature importances
importances = best_rf.feature_importances_

# Convert the importances into one-dimensional 1darray with corresponding df column names as axis labels
f_importances = pd.Series(importances, train_df.columns)

# Sort the array in descending order of the importances
f_importances.sort_values(ascending=False, inplace=True)

# Make the bar Plot from f_importances
f_importances.plot(x='Features', y='Importance', kind='bar', figsize=(16,9), rot=45, fontsize=15)

# Show the plot
plt.tight_layout()
plt.savefig('feature_importances.jpeg', dpi=300)
plt.show()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(features_df, target_df, test_size=0.2, random_state=42)

# Train the model
best_rf.fit(X_train, y_train)

# Make predictions
predictions = best_rf.predict(X_test)

Random Forest Actual VS Predicted Values

In [ ]:
# Make predictions on the correct data
predictions = best_rf.predict(X_test)

# Calculate the error
errors = y_test - predictions  # If 'y_test' is a Series

# Create a figure and axis
fig, ax = plt.subplots()

# Create a scatter plot of actual vs predicted values
ax.scatter(y_test, predictions, edgecolors=(0, 0, 0))

# Add a line for perfect prediction
ax.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'k--', lw=4)

# Set labels
ax.set_xlabel('Actual')
ax.set_ylabel('Predicted')
ax.set_title('Actual vs Predicted Values')

# Show the plot
plt.savefig('random_forest_actual_vs_predicted.jpeg', dpi=300)
plt.show()


Random Forest Histogram of Prediction Errors

In [ ]:
# Create a figure and axis for error plot
fig, ax = plt.subplots()

# Create a histogram of the error
ax.hist(errors, bins=20)

# Set labels
ax.set_xlabel('Prediction Error')
ax.set_ylabel('Frequency')
ax.set_title('Random Forest Prediction Error')

# Show the plot
plt.savefig('random_forest_prediction_errors.jpeg', dpi=300)
plt.show()